<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/CST3512_SEC_Edgar_yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SEC Edgar** and `yfinance` for **Yahoo! Finance**         

[per Wikipedia](https://en.wikipedia.org/wiki/EDGAR):    

---

"**EDGAR, the Electronic Data Gathering, Analysis, and Retrieval** system, performs automated collection, validation, indexing, acceptance, and forwarding of submissions by companies and others who are required by law to file forms with the U.S. Securities and Exchange Commission (the "SEC"). The database contains a wealth of information about the Commission and the securities industry which is freely available to the public via the Internet."     

*This notebook uses the [sec-edgar-downloader repository by jadchaar on Github](https://github.com/jadchaar/sec-edgar-downloader) to demonstrate an approach to accessing the wealth of data available on the SEC Edgar platform, which is based on [XBRL](https://www.xbrl.org/) (a form of XML) transmission of data.*    


---    

"**Yahoo! Finance** is a media property that is part of the **Yahoo!** network. It provides financial news, data and commentary including stock quotes, press releases, financial reports, and original content. It also offers some online tools for personal finance management. In addition to posting partner content from other web sites, it posts original stories by its team of staff journalists. It is ranked 21st by SimilarWeb on the list of largest news and media websites.    

**Yahoo! Finance** recently added the feature to look at news surrounding cryptocurrency. It lists over 9,000 unique coins including Bitcoin and Ethereum."    

*This notebook uses the [**yfinance** module by ranaroussi on Github](https://github.com/ranaroussi/yfinance) to demonstrate an approach to accessing many different types of financial statement and stock/coin trading data available on the **Yahoo! Finance** platform.*   





---



##Contents    


[sec-edgar-downloader](https://github.com/jadchaar/sec-edgar-downloader) is a Python package for downloading company filings from the SEC EDGAR database. Searches can be conducted either by **stock ticker** or **Central Index Key (CIK)**. You can use the [SEC CIK lookup tool](https://www.sec.gov/edgar/searchedgar/cik.htm) if you cannot find an appropriate ticker.


> Lookup up stock ticker symbols using a company name search on [Yahoo! Finance](https://finance.yahoo.com/lookup) or by using the [search page on NasdaqTrader.com.](https://www.nasdaqtrader.com/trader.aspx?id=symbollookup)



This notebook begins with two sections on accessing SEC Edgar data:    
1. **Basic Usage** -- to use module imports and some straightforward code to do simple lookups    
2. **Advanced Usage** -- alternate code and approach for more complex/advanced queries and document lookups    

This notebook also has a third section on using the [**yfinance** module by ranaroussi on Github](https://github.com/ranaroussi/yfinance) to access ticker data from Yahoo Finance:    
3. **yfinance** -- for information, historical market data, financial statements, analysts recommendations, option chains, and more on publicly traded stocks     

Finally, there is an appendix in this notebook to describe the structure of `XBRL .xml` documents along with an example of parsing `XBRL .xml` with BeautifulSoup.  There are other methods for automatically extracting data from `XBRL .xml` documents such as the `python-xbrl` library.
4. **APPENDIX: Parsing XBRL with Python** -- by Matt Scarpino, 02-Feb-2018 [blog post on CodeProject](https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python)     

*please see the footnotes at the bottom of this notebook for disclaimers*    

##Housekeeping    

For the `sec-edgar-downloader` Basic and Advanced scripts, be sure to import the following module:    

* from sec_edgar_downloader import Downloader    

The `sec-edgdar-downloade` module can be imported using the following statement:    
* `!pip install -U sec-edgar-downloader`    


For the `yfinance` scripts, be sure to import the following module:     
* import yfinance as yf    

The `yfinance` module can be imported using the following statement:    
* `!pip install yfinance --upgrade --no-cache-dir`    

To use a custom requests session with `yfinance` (for example to cache calls to the API or customize the User-agent header), pass a `session= argument` to the Ticker constructor as in the following code:    


```
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'
ticker = yf.Ticker('msft aapl goog', session=session)
# The scraped response will be stored in the cache
ticker.actions
```

And, if you intend to work with the example of parsing `XBRL .xml` files in the **APPENDIX: Parsing XBRL with Python**, then you will want to import `BeautifulSoup`, `requests`, and `sys` as in the following code:    


```
from bs4 import BeautifulSoup
import requests
import sys
```



In [1]:
# importing sec-edgar-downloader for access to SEC Edgar data    
!pip install -U sec-edgar-downloader    
from sec_edgar_downloader import Downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.4 MB/s 


In [2]:
# importing yfinance as yf for access to yahoo! Finance data    
!pip install yfinance --upgrade --no-cache-dir    
import yfinance as yf 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 6.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [1]:
# importing beautifulsoup, requests, and sys for XBRL parsing example in the APPENDIX
from bs4 import BeautifulSoup
import requests
import sys

In [2]:
# import prettyprint (pprint) for easy-to-read output 
import pprint 



---



##1. Basic SEC Edgar (downloader) Usage     


This section demonstrates downloads of `HTML` documents for various SEC Edgar filings on publicly traded companies.    

The official list of all SEC forms (documents) can be found on the [U.S. Securities and Exchange Commission - Forms List](https://www.sec.gov/forms).    

Explanations of the content and use of some of the more frequently analyzed SEC Edgar submission forms is available on [Investopedia - SEC Filings: Forms You Need to Know page](https://www.investopedia.com/articles/fundamental-analysis/08/sec-forms.asp#:~:text=Among%20the%20most%20common%20SEC,114%2C%20and%20Foreign%20Investment%20Disclosures.).     

In [3]:
from sec_edgar_downloader import Downloader

# Initialize a downloader instance. If no argument is passed
# to the constructor, the package will download filings to
# the current working directory.
# dl = Downloader("/path/to/valid/save/location") # to specify a path
dl = Downloader()  # to use the current working directory   

# Get all 8-K filings for Apple (ticker: AAPL)
# dl.get("8-K", "AAPL")  # this is lengthy

# Get all 8-K filings for Apple, including filing amends (8-K/A)
# dl.get("8-K", "AAPL", include_amends=True) # this is lengthy

# Get all 8-K filings for Apple after January 1, 2017 and before March 25, 2017
# Note: after and before strings must be in the form "YYYY-MM-DD"
dl.get("8-K", "AAPL", after="2017-01-01", before="2017-03-25") # this is brief

# Get the five most recent 8-K filings for Apple
dl.get("8-K", "AAPL", amount=5) # this is brief

# Get all 10-K filings for Microsoft
# dl.get("10-K", "MSFT")  # this is moderate in length

# Get the latest 10-K filing for Microsoft
dl.get("10-K", "MSFT", amount=1) # this is brief 

# Get all 10-Q filings for Visa
# dl.get("10-Q", "V") # this is very lengthy 

# Get all 13F-NT filings for the Vanguard Group
dl.get("13F-NT", "0000102909")  # this is brief

# Get all 13F-HR filings for the Vanguard Group
# dl.get("13F-HR", "0000102909")  # this is lengthy 

# Get all SC 13G filings for Apple
dl.get("SC 13G", "AAPL") # this is brief

# Get all SD filings for Apple
dl.get("SD", "AAPL") # this is brief 

9

Let's look at SEC Edgar filings regarding insider trading of **Twitter (`TWTR`)**  stock.    

We can wrap the `dl.get()` method with a `print()` statement to display a count of the number of documents downloaded.    


In [4]:
# from sec_edgar_downloader import Downloader

# Initialize a downloader instance. If no argument is passed
# to the constructor, the package will download filings to
# the current working directory.
# dl = Downloader("/path/to/valid/save/location") # to specify a path
# dl = Downloader()  # to use the current working directory   

# Get the last 6 filings of Form-3 (Beneficial Ownership) for Twitter (ticker: TWTR)
print('Form 3 downloads', dl.get("3", "TWTR", amount=6))  # Is Elon Musk on any of these?

# Get the last six month's worth of Form-4 (Changes in Ownership) for Twitter (ticker: TWTR)
print('Form 4 downloads', dl.get("4", "TWTR", after="2021-10-31", before="2022-05-01"))  # Is Elon Musk on any of these?

# Get all of the filings of Form-5 (Annual Summary of #4) for Twitter (ticker: TWTR)
print('Form 5 downloads', dl.get("5", "TWTR"))  # Is Elon Musk on any of these?


Form 3 downloads 6
Form 4 downloads 42
Form 5 downloads 0




---



##2. Advanced SEC Edgar (downloader) Usage     



While this section claims to support more advanced uses of SEC Edgar data, like the basic section, it is focused on downloading documents and not on parsing the data in those documents.  The more advanced features described in this section are somewhat more complex search terms for the selection of a document(s) to be downloaded.    


In [ ]:
from sec_edgar_downloader import Downloader

# Download filings to the current working directory
dl = Downloader()

# Get all Apple proxy statements that contain the term "antitrust"
dl.get("DEF 14A", "AAPL", query="antitrust")

# Get all 10-K filings for Microsoft without the filing details
dl.get("10-K", "MSFT", download_details=False)

# Get the latest supported filings, if available, for Apple
for filing_type in dl.supported_filings:
    dl.get(filing_type, "AAPL", amount=1)

# Get the latest supported filings, if available, for a
# specified list of tickers and CIKs
equity_ids = ["AAPL", "MSFT", "0000102909", "V", "FB"]
for equity_id in equity_ids:
    for filing_type in dl.supported_filings:
        dl.get(filing_type, equity_id, amount=1)    


###Supported SEC Document File Types (forms)    

The `sec-edgar-downloader` package supports downloading all SEC filing types (6-K, 8-K, 10-K, DEF 14A, S-1, and many others). You can learn more about the different SEC filing types [here](https://www.investopedia.com/articles/fundamental-analysis/08/sec-forms.asp)).     

Below is an exhaustive list of all filings types that can be downloaded by the `sec-edgar-downloader` package:     

*(explode the following cell for a detailed list of document form names)*

#### `sec-edgar-downloader` supported document form names 

* 1     
* 1-A     
* 1-A POS     
* 1-A-W     
* 1-E     
* 1-E AD     
* 1-K     
* 1-SA     
* 1-U     
* 1-Z     
* 1-Z-W     
* 10-12B     
* 10-12G     
* 10-D     
* 10-K     
* 10-KT     
* 10-Q     
* 10-QT     
* 11-K     
* 11-KT     
* 13F-HR     
* 13F-NT     
* 13FCONP     
* 144     
* 15-12B     
* 15-12G     
* 15-15D     
* 15F-12B     
* 15F-12G     
* 15F-15D     
* 18-12B     
* 18-K     
* 19B-4E     
* 2-A     
* 2-AF     
* 2-E     
* 20-F     
* 20FR12B     
* 20FR12G     
* 24F-2NT     
* 25     
* 25-NSE     
* 253G1     
* 253G2     
* 253G3     
* 253G4     
* 3     
* 305B2     
* 34-12H     
* 4     
* 40-17F1     
* 40-17F2     
* 40-17G     
* 40-17GCS     
* 40-202A     
* 40-203A     
* 40-206A     
* 40-24B2     
* 40-33     
* 40-6B     
* 40-8B25     
* 40-8F-2     
* 40-APP     
* 40-F     
* 40-OIP     
* 40FR12B     
* 40FR12G     
* 424A     
* 424B1     
* 424B2     
* 424B3     
* 424B4     
* 424B5     
* 424B7     
* 424B8     
* 424H     
* 425     
* 485APOS     
* 485BPOS     
* 485BXT     
* 486APOS     
* 486BPOS     
* 486BXT     
* 487     
* 497     
* 497AD     
* 497H2     
* 497J     
* 497K     
* 5     
* 6-K     
* 6B NTC     
* 6B ORDR     
* 8-A12B     
* 8-A12G     
* 8-K     
* 8-K12B     
* 8-K12G3     
* 8-K15D5     
* 8-M     
* 8F-2 NTC     
* 8F-2 ORDR     
* 9-M     
* ABS-15G     
* ABS-EE     
* ADN-MTL     
* ADV-E     
* ADV-H-C     
* ADV-H-T     
* ADV-NR     
* ANNLRPT     
* APP NTC     
* APP ORDR     
* APP WD     
* APP WDG     
* ARS     
* ATS-N     
* ATS-N-C     
* ATS-N/UA     
* AW     
* AW WD     
* C     
* C-AR     
* C-AR-W     
* C-TR     
* C-TR-W     
* C-U     
* C-U-W     
* C-W     
* CB     
* CERT     
* CERTARCA     
* CERTBATS     
* CERTCBO     
* CERTNAS     
* CERTNYS     
* CERTPAC     
* CFPORTAL     
* CFPORTAL-W     
* CORRESP     
* CT ORDER     
* D     
* DEF 14A     
* DEF 14C     
* DEFA14A     
* DEFA14C     
* DEFC14A     
* DEFC14C     
* DEFM14A     
* DEFM14C     
* DEFN14A     
* DEFR14A     
* DEFR14C     
* DEL AM     
* DFAN14A     
* DFRN14A     
* DOS     
* DOSLTR     
* DRS     
* DRSLTR     
* DSTRBRPT     
* EFFECT     
* F-1     
* F-10     
* F-10EF     
* F-10POS     
* F-1MEF     
* F-3     
* F-3ASR     
* F-3D     
* F-3DPOS     
* F-3MEF     
* F-4     
* F-4 POS     
* F-4MEF     
* F-6     
* F-6 POS     
* F-6EF     
* F-7     
* F-7 POS     
* F-8     
* F-8 POS     
* F-80     
* F-80POS     
* F-9     
* F-9 POS     
* F-N     
* F-X     
* FOCUSN     
* FWP     
* G-405     
* G-405N     
* G-FIN     
* G-FINW     
* IRANNOTICE     
* MA     
* MA-A     
* MA-I     
* MA-W     
* MSD     
* MSDCO     
* MSDW     
* N-1     
* N-14     
* N-14 8C     
* N-14MEF     
* N-18F1     
* N-1A     
* N-2     
* N-23C-2     
* N-23C3A     
* N-23C3B     
* N-23C3C     
* N-2MEF     
* N-30B-2     
* N-30D     
* N-4     
* N-5     
* N-54A     
* N-54C     
* N-6     
* N-6F     
* N-8A     
* N-8B-2     
* N-8F     
* N-8F NTC     
* N-8F ORDR     
* N-CEN     
* N-CR     
* N-CSR     
* N-CSRS     
* N-MFP     
* N-MFP1     
* N-MFP2     
* N-PX     
* N-Q     
* NO ACT     
* NPORT-EX     
* NPORT-NP     
* NPORT-P     
* NRSRO-CE     
* NRSRO-UPD     
* NSAR-A     
* NSAR-AT     
* NSAR-B     
* NSAR-BT     
* NSAR-U     
* NT 10-D     
* NT 10-K     
* NT 10-Q     
* NT 11-K     
* NT 20-F     
* NT N-CEN     
* NT N-MFP     
* NT N-MFP1     
* NT N-MFP2     
* NT NPORT-EX     
* NT NPORT-P     
* NT-NCEN     
* NT-NCSR     
* NT-NSAR     
* NTFNCEN     
* NTFNCSR     
* NTFNSAR     
* NTN 10D     
* NTN 10K     
* NTN 10Q     
* NTN 20F     
* OIP NTC     
* OIP ORDR     
* POS 8C     
* POS AM     
* POS AMI     
* POS EX     
* POS462B     
* POS462C     
* POSASR     
* PRE 14A     
* PRE 14C     
* PREC14A     
* PREC14C     
* PREM14A     
* PREM14C     
* PREN14A     
* PRER14A     
* PRER14C     
* PRRN14A     
* PX14A6G     
* PX14A6N     
* QRTLYRPT     
* QUALIF     
* REG-NR     
* REVOKED     
* RW     
* RW WD     
* S-1     
* S-11     
* S-11MEF     
* S-1MEF     
* S-20     
* S-3     
* S-3ASR     
* S-3D     
* S-3DPOS     
* S-3MEF     
* S-4     
* S-4 POS     
* S-4EF     
* S-4MEF     
* S-6     
* S-8     
* S-8 POS     
* S-B     
* S-BMEF     
* SC 13D     
* SC 13E1     
* SC 13E3     
* SC 13G     
* SC 14D9     
* SC 14F1     
* SC 14N     
* SC TO-C     
* SC TO-I     
* SC TO-T     
* SC13E4F     
* SC14D1F     
* SC14D9C     
* SC14D9F     
* SD     
* SDR     
* SE     
* SEC ACTION     
* SEC STAFF ACTION     
* SEC STAFF LETTER     
* SF-1     
* SF-3     
* SL     
* SP 15D2     
* STOP ORDER     
* SUPPL     
* T-3     
* TA-1     
* TA-2     
* TA-W     
* TACO     
* TH     
* TTW     
* UNDER     
* UPLOAD     
* WDL-REQ     
* X-17A-5     




---



##3. `yfinance` for Yahoo! Finance     



`yfinance` offers a threaded and Pythonic way to download market data from **Yahoo! Finance**    

**Legal note:**    

**Yahoo!, Y!Finance, and Yahoo! Finance** are registered trademarks of Yahoo, Inc.

`yfinance` is not affiliated, endorsed, or vetted by Yahoo, Inc. It's an open-source tool that uses Yahoo's publicly available APIs, and is intended for research and educational purposes.

You should refer to Yahoo!'s terms of use ([here](https://policies.yahoo.com/us/en/yahoo/terms/product-atos/apiforydn/index.htm), [here](https://legal.yahoo.com/us/en/yahoo/terms/otos/index.html), and [here](https://policies.yahoo.com/us/en/yahoo/terms/index.htm)) for details on your rights to use the actual data downloaded. **Remember - the Yahoo! Finance API is intended for personal use only.**    


###Introducing the `Ticker()` module:    

The `Ticker()` module allows you get market and meta data for a security, using a Pythonic way:    


In [6]:
import yfinance as yf

msft = yf.Ticker("MSFT")
print(msft)


yfinance.Ticker object <MSFT>


In [7]:
# get stock info
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise

In [8]:
# get historical market data
msft.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.055380,0.063524,0.055380,0.060809,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.060809,0.064067,0.060809,0.062980,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062980,0.064609,0.062980,0.064067,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.064067,0.064610,0.061894,0.062437,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.062437,0.062980,0.060809,0.061351,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2022-11-22 00:00:00-05:00,243.589996,245.309998,240.710007,245.029999,19665700,0.0,0.0
2022-11-23 00:00:00-05:00,245.110001,248.279999,244.270004,247.580002,19508500,0.0,0.0
2022-11-25 00:00:00-05:00,247.309998,248.699997,246.729996,247.490005,9200800,0.0,0.0


In [9]:
# show dividends
msft.dividends

Date
2003-02-19 00:00:00-05:00    0.08
2003-10-15 00:00:00-04:00    0.16
2004-08-23 00:00:00-04:00    0.08
2004-11-15 00:00:00-05:00    3.08
2005-02-15 00:00:00-05:00    0.08
                             ... 
2021-11-17 00:00:00-05:00    0.62
2022-02-16 00:00:00-05:00    0.62
2022-05-18 00:00:00-04:00    0.62
2022-08-17 00:00:00-04:00    0.62
2022-11-16 00:00:00-05:00    0.68
Name: Dividends, Length: 76, dtype: float64

In [10]:
# show splits
msft.splits

Date
1987-09-21 00:00:00-04:00    2.0
1990-04-16 00:00:00-04:00    2.0
1991-06-27 00:00:00-04:00    1.5
1992-06-15 00:00:00-04:00    1.5
1994-05-23 00:00:00-04:00    2.0
1996-12-09 00:00:00-05:00    2.0
1998-02-23 00:00:00-05:00    2.0
1999-03-29 00:00:00-05:00    2.0
2003-02-18 00:00:00-05:00    2.0
Name: Stock Splits, dtype: float64

Available paramaters for the history() method are:    

* **period:** data period to download (Either Use period parameter or use start and end) Valid periods are: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max    
* **interval:** data interval (intraday data cannot extend last 60 days) Valid intervals are: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo    
* **start:** If not using period - Download start date string (YYYY-MM-DD) or datetime.    
* **end:** If not using period - Download end date string (YYYY-MM-DD) or datetime.    
* **prepost:** Include Pre and Post market data in results? (Default is False)    
* **auto_adjust:** Adjust all OHLC automatically? (Default is True)
* **actions:** Download stock dividends and stock splits events? (Default is True)    


###Mass download of market data:    

You can also download data for multiple tickers at once, like before.    
  

In [ ]:
import yfinance as yf
data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

To access the closing price data for **SPY**, you should use: `data['Close']['SPY']`.    

If, however, you want to group data by **Symbol**, use:   
  

In [ ]:
import yfinance as yf
data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30",
                   group_by="ticker") 


To access the closing price data for **SPY**, you should use: `data['SPY']['Close']`.

The `.download()` method accepts an additional parameter - threads for faster completion when downloading a lot of symbols at once.

*NOTE: To keep compatibility with older versions, auto_adjust defaults to False when using mass-download.*  
  


### Using pandas_datareader:    

If your legacy code is using `pandas_datareader` and you wand to keep the code changes to minimum, you can simply call the override method and keep your code as it was:  
  

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe using pandas_datareader
data = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-04-30") 




---



##4. APPENDIX: Parsing XBRL with Python     

from the [02-Feb-2018 blog post by Dan Scavino on CodeProject](https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python)     



So far this notebook has demonstrated how to access corporate reports in the EDGAR database, but it didn't explain how to extract data from a report. If you look at a report listing, you'll see that EDGAR provides reports in three primary formats:    

* **Regular text** - Data provided in regular files (*.txt)    
* **Web pages** - Data to be viewed in a browser (*.htm)    
* **XBRL** - Data provided in XBRL-formatted files (*.xml)    

The first two options are fine if you want to read report data yourself. But if you want to extract data programmatically, the last option is the most practical. XBRL files aren't easy for humans to read, but because of their structure, they're ideally suited for computers.    

This appendix describes the XBRL format and then explains how to read XBRL using BeautifulSoup. At the end, I'll present example code that programmatically downloads and parses an XBRL file from EDGAR.    

###4.1. Introducing XBRL    

A primary role of the **US Securities and Exchange Commission (SEC)** is to ensure that investors have reliable information with which to make decisions. To this end, the **SEC** requires that publicly-traded corporations submit reports that accurately portray their financial state. Corporations have traditionally provided these reports in regular text, but as computerized stock analysis became popular, the **SEC** decided on a more structured, computer-readable format.    

The **SEC** selected the **eXtensible Business Reporting Language** (`XBRL`) for structured corporate reporting. As of April 2009, the **SEC** requires that corporations provide financial reports in `XBRL` format in addition to text. Since then, India and the United Kingdom have also adopted `XBRL` for corporate reporting.    

`XBRL` is based on the **eXtensible Markup Language** (`XML`), but uses special tags to mark financial data. This section presents the basics of `XML` and namespaces, and then provides an overview of `XBRL`.    


####4.1.a XML, Schema, and Namespaces    

A good way to introduce `XML` is to compare it with HTML. An `HTML` document structures its content using nested tags that take the form `<xyz>...</xyz>`. For example, `HTML` uses `<b>...</b>` tags to display text in boldface, as in `<b>Hi there!</b>`. HTML lets you control a tag's behavior with attributes, such as the id attribute in `<p id="...">...</p>`.    

I like to think of `XML` as generic `HTML`. An `XML` document contains tags and attributes similar to those in `HTML` but `XML` doesn't define any specific tags or attributes. Instead, implementers can define their own tags and attributes by creating a schema. Schemas are defined in special `XML` documents formatted with **XML Schema Definition (XSD)**, and for this reason, schema documents have the suffix `*.xsd` instead of `*.xml`.    

An XML document can access the tags and attributes of a schema using a namespace declaration. As an example, the following declaration specifies that the `XML` document will access the tags and attributes defined in the schema located at `http://www.example.com`:    

In [ ]:
# sample XML 
xmlns:ex="http://www.example.com"

####4.1.b XBRL Reports and Schema    

An `XBRL` document is an `XML` document that structures its content using `XBRL`'s tags and attributes. This may sound straightforward, but a single document may need to access features from many different schemas. For example, different countries have different reporting requirements, so an American report will access a different set of elements than a British report. Similarly, different types of reports will require different schemas, so an annual report will use different tags than a prospectus.    

A thorough discussion of the tags/attributes in an American corporation's annual report would take up a sizable book. In this discussion, my goal is to present some of the namespaces that are commonly accessed in American reports:    

* **Base XBRL Schema** - Provides the overall structure of an `XBRL` document    
* **US Document and Entity Information (DEI)** - Sets a document's type and characteristics    
* **US Generally Accepted Accounting Principles (GAAP)** - Defines required elements of American reports    
* **Entity-specific Schema** - Defines elements specific to the entity providing the report    

You don't need to memorize the elements of these namespaces, but the more familiar you are, the better you'll be able to extract data from `XBRL` documents.    


#####4.1.b.i The Base XBRL Schema     

The fundamental tags and attributes of `XBRL` are provided in the schema located at *http://www.xbrl.org/2003/instance*. Documents commonly access these elements through the `xbrli` prefix, as given in the following namespace declaration:    



In [ ]:
# XML example 
xlmns:xbrli="http://www.xbrl.org/2003/instance"    

Of the many elements defined by the schema, `xbrli:xbrl` is particularly important. This is because the content of every XBRL document must be contained inside `<xbrli:xbrl>...</xbrli:xbrl>` tags.    

To understand other tags provided by the base schema, you should be familiar with the following terms:  

* **instance** - an `XBRL` document whose root element is <xbrli:xbrl>   
* **fact** - an individual detail in a report, such as $20M   
* **concept** - the meaning associated with a fact, such as the cost of goods sold   
* **entity** - the company or individual described by a concept    
* **context** - a data structure that associates an entity with a concept    

Many `XBRL` documents start by defining a long list of contexts. Each context is represented by an `<xbrli:context>` element and each has an id attribute. Each `<xbrli:context>` element contains an `<xbrli:entity>` subelement that identifies an entity. The following markup defines a context with an identifier of **FD2013Q4YTD**:     



In [ ]:
# XML example    
<xbrli:context id="FD2013Q4YTD">
    <xbrli:entity>
    <xbrli:identifier scheme="http://www.sec.gov/CIK">0001065088</xbrli:identifier>
  </xbrli:entity>
  <xbrli:period>
    <xbrli:startDate>2013-01-01</xbrli:startDate>
    <xbrli:endDate>2013-12-31</xbrli:endDate>
  </xbrli:period>
</xbrli:context> 


Later sections in the document can reference this context by assigning a contextRef attribute to the context's ID. This is shown in the following markup:   


In [ ]:
# XML example 
<us-gaap:IncomeTaxDisclosureTextBlock contextRef="FD2013Q4YTD" ...>


####4.1.b.ii US Document and Entity Information (DEI)    

Every `XBRL` document submitted to the **SEC** needs to provide information about its content. A submitter can meet this requirement by including elements from the **US Document and Entity Information (DEI)** schema. These elements are commonly prefixed with `dei` and a document can access them with the following declaration:    
*italicized text*

In [ ]:
# XML example
xlmns:dei="http://xbrl.sec.gov/dei/2014-01-31"

The elements defined in this schema identify the XBRL report's type and provide information about the entity submitting the report. *Table 1* lists eleven of the many elements available.    

**Table 1: Elements Provided by the US Document and Entity Information Schema (Abridged)**    


* **DocumentType** >> Type of document being reported    
* **EntityCentralIndexKey** >> CIK of the entity submitting the report    
* **TradingSymbol** >> Exchange symbol of the entity submitting the report    
* **EntityCurrentReportingStatus** >> Identifies if the entity is subject to filing requirements    
* **EntityFilerCategory** >> Identifies the entity's filing category (large, small, ...    
* **EntityRegistrantName** >> Exact name of the entity has given in the charter    
* **DocumentFiscalPeriodFocus** >> The document's focus fiscal period    
* **DocumentFiscalYearFocus** >> The document's focus fiscal year    
* **CurrentFiscalYearEndDate** >> End of the current fiscal year    
* **AmendmentFlag** >> Identifies if the document is an amendment to a
previously-filed document    
* **AmendmentDescription**>>Description of changes in amended document    



It's important to see the difference between `EntityCentralIndexKey`, `TradingSymbol`, and `EntityRegistrantName`. The `EntityCentralIndexKey` element identifies the submitter's **CIK code**, the `TradingSymbol` identifies the submitter's **trading (ticker) symbol**, and `EntityRegistrantName` provides the entity's formal name.

The following markup, taken from an **eBay annual report**, demonstrates how DEI elements are used:    


In [ ]:
# XML example    
<dei:DocumentType contextRef="..." id="Fact-...">
  10-K
</dei:DocumentType>
<dei:EntityCentralIndexKey contextRef="..." id="Fact-...">
  0001065088
</dei:EntityCentralIndexKey>
<dei:TradingSymbol contextRef="..." id="Fact-...">
  EBAY
</dei:TradingSymbol>
<dei:EntityRegistrantName contextRef="..." id="Fact-...">
  EBAY INC
</dei:EntityRegistrantName>
<dei:EntityFilerCategory contextRef="..." id="Fact-...">
  Large Accelerated Filer
</dei:EntityFilerCategory> 


As shown, each `DEI` element has an id attribute and a contextRef that refers to an `<xbrli:context>` element defined earlier in the document.    


####4.1.b.iii US Generally Accepted Accounting Principles (GAAP)

To ensure that businesses use common terminology in their accounting reports, the **US Financial Accounting Standards Board (FASB)** provides a set of standards called the **Generally Accepted Accounting Principles**, or `GAAP`. Entities can provide `GAAP` data in their `XBRL` reports by accessing the **FASB**'s schema definitions. `GAAP` elements are commonly preceded with the `us-gaap` prefix:    



In [ ]:
# XML Example 
xmlns:us-gaap="http://fasb.org/us-gaap/2014-01-31"

This schema provides thousands of elements related to accounting, and *Table 2* lists a small but important subset. You can look through a more complete table [here](http://www.xbrlsite.com/LinkedData/BrowseObjectsByType_HTML.aspx?Type=%5BConcept%5D&Submit=Submit).   


**Table 2: Elements of the US Generally Accepted Accounting Principles Schema (Abridged)**     

**AccountsPayableCurrent** >> Liabilities payable to vendors as of the balance sheet date    
**AccountsReceivableGross** >> Amounts due from customers or clients    
**AccountsReceivableNet** >> Amounts due from customers or clients, reduced to estimated realizable value    
**AccruedIncomeTaxes** >> Unpaid sum of known and estimated tax obligations    
**AccruedInsuranceCurrent** >> Obligations payable to insurance entities to mitigate loss    
**AssetManagementCosts** >> Aggregate costs related to asset management    
**AssetsCurrent** >> Sum of all assets expected to be realized within year    
**BorrowedFunds** >> Sum of all debt amounts    
**Cash** >> Unrestricted cash available for operating needs    
**CommercialPaper** >> Value of short-term borrowings using unsecured
obligations issued by banks and corporations    
**CommonStockNoParValue** >> Issuance value per share of no-par value stock    
**CommonStockSharesIssued** >> Total number of common shares that have been
sold or granted to shareholders    
**CommonStockValue** >> Aggregate par or stated value of issued common stock    
**SalariesAndWages** >> Expenditures for salaries other than officers    
**ConvertibleDebt** >> Amount of debt that can be converted into another
form of financial instrument, such as common stock    
**CostOfGoodsSold** >> Aggregate costs related to goods sold during the period    
**CostOfServices** >> Total costs related to services rendered during the period    
**CostsAndExpenses** >> Total costs of sales and operating expenses for the period    
**DebtCurrent** >> Sum of short-term debt and maturities of long-term debt    
**DeferredRevenue** >> Cash or other assets that have not yet been realized    
**Depreciation** >> Amount of expense related to the cost of tangible assets
over the assets' useful lives    
**DirectOperatingCosts** >> Aggregate expenses directly related to operations    
**Dividends** >> Equity impact of cash, stock, and dividends declared
for all securities during the period    
**EarningsPerShareBasic** >> Net income (loss) for the period per share of common stock    
**GrossProfit** >> Aggregate revenue minus the cost of goods/services sold and operating expenses    
**IntangibleAssetsCurrent** >> Current portion of non-physical assets, excluding financial assets    
**InterestAndDebtExpense** >> Expenses related to interest and debt payments    
**InventoryGross** >> Merchandise, goods, or supplies held for future sale or used in manufacturing or production    
**Land** >> Real estate held for productive use, not held for sale    
**Liabilities** >> Sum of all recognized liabilities    
**LiabilitiesAndStockholdersEquity** >> Total of liabilities and stockholder's equity, including the portion of equity attributable to noncontrolling interests    
**NetIncomeLoss** >> Portion of profit or loss for the period, net of income taxes    
**ProfitLoss** >> Consolidated profit or loss for the period    
**NotesPayable** >> Aggregate amount of notes payable, with initial maturities beyond one year or the normal operating cycle    
**OfficersCompensation** >> Expenditures for salaries of officers    
**OperatingCycle** >> Entity's operating cycle if less than 12 months    
**OperatingExpenses** >> Recurring costs associated with normal operations except expenses included in the cost of sales or services    
**PreferredStockValue** >> Stated value of issued nonredeemable preferred stock    
**ResearchAndDevelopmentExpense** >> Costs incurred during research and development activities    
**Revenues** >> Aggregate revenue recognized during the period    
**SharesIssued** >> Number of shares of stock issued    
**SharesOutstanding** >> Number of shares issued and outstanding    
**StockholdersEquity** >> Total of stockholders' equity items, net of receivables from officers, directors, owners, and affiliates    

You can find accounting data in a report by searching for the appropriate `us-gaap` element. For example, **eBay's 2014** annual report identifies its **aggregate liabilities** with the following markup:   

<center><b><u>XBRL Example XML Content</u></b></center>

```
<us-gaap:Liabilities contextRef="..." decimals="..." id="..." unitRef="usd">
  25226000000
</us-gaap:Liabilities> 
```

The `us-gaap` schema has many elements that closely resemble one another in name and purpose. If you're searching for specific accounting data, be sure not to confuse the elements.

###4.2. Parsing XBRL with BeautifulSoup    

After you've downloaded an XBRL document, you can extract its data using a number of methods. If you know what element you're interested in, you can perform a brute-force search for the text, as in `us-gaap:Assets`. At the opposite extreme, the `python-xbrl` library was specially created for parsing `XBRL` documents, but I've never gotten it to work properly.    

This section explains how to parse `XBRL` using the `BeautifulSoup` package. You don't need to learn any new classes or methods, but it is important to specify that you want to perform `XML` parsing. If you install the `lxml` library (`pip install lxml`), then you can create the `BeautifulSoup` instance with the following code:    


In [ ]:
soup = BeautifulSoup(..., 'lxml')

For some reason, when I call the `find_all` method to search for an `XBRL` tag, the returned list is always empty. But when I call `find_all` without arguments, the returned list contains Tags that represent `XBRL` tags. Therefore, I use code like the following:    


In [ ]:
soup = BeautifulSoup(xbrl_string, 'lxml')
tag_list = soup.find_all()
for tag in tag_list:
    if tag.name == 'us-gaap:liabilities':
        print('Liabilities: ' + tag.text)    


An annual report may contain multiple `<us-gaap:liabilities>` elements, each corresponding to a different reporting period. Each period corresponds to a `<context>` element, so you can distinguish between `GAAP` elements by checking their `contextRef` attributes.    


##4.3.a Complete EDGAR-XBRL Example (4.3.a needs work -- see 4.3.b)

If you are familira with BeautifulSoup and the content of this article, you shouldn't have any trouble understanding how to access a company's EDGAR reports and parse them in Python. To demonstrate this, the code in *Listing 1* searches EDGAR for the 2014 annual report (10-K) from IBM (CIK: 0000051143) and then parses the XBRL to determine the stockholder's equity (`us-gaap:stockholdersequity`),   


###**THE FOLLOWING CODE NEEDS TO BE UPDATED**

**Listing 1: Reading Stockholder's Equity from IBM's Annual Report (xbrl_reader.py)**

In [ ]:
from bs4 import BeautifulSoup
import requests
import sys

# Access page
cik = '0000051143'
type = '10-K'
dateb = '20160101'

# Obtain HTML for search page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
edgar_resp = requests.get(base_url.format(cik, type, dateb))
edgar_str = edgar_resp.text

# Find the document link
doc_link = ''
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if '2015' in cells[3].text:
            doc_link = 'https://www.sec.gov' + cells[1].a['href']

# Exit if document link couldn't be found
if doc_link == '':
    print("Couldn't find the document link")
    # sys.exit() # not worth trying to exit() in Colab!

# Obtain HTML for document page
doc_resp = requests.get(doc_link)
doc_str = doc_resp.text

# Find the XBRL link
xbrl_link = ''
soup = BeautifulSoup(doc_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile', summary='Data Files')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if 'INS' in cells[3].text:
            xbrl_link = 'https://www.sec.gov' + cells[2].a['href']

# Obtain XBRL text from document
xbrl_resp = requests.get(xbrl_link)
xbrl_str = xbrl_resp.text

# Find and print stockholder's equity
soup = BeautifulSoup(xbrl_str, 'lxml')
tag_list = soup.find_all()
for tag in tag_list:
    if tag.name == 'us-gaap:stockholdersequity':
        print("Stockholder's equity: " + tag.text)



---



##4.3.b `python-xbrl` Simple XBRL Parsing Example     

See the [`python-xbrl` documentation](https://pypi.org/project/python-xbrl/) for more information.    

*note that [there are some dangling segments of code in the `python-xbrl` library](https://stackoverflow.com/questions/33963598/parsing-xbrl-using-python) which does does not appear to have been actively maintained over the years*

*note: this section uses the following example `XBRL .xml` files:* 

* sam-20131228.xml    
* sam-20140927.xml    

*each file may be accessed from Professor Patrick's Github using the following statements, respectively:*    


```
!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/sam-20131228.xml -o "sam-20131228.xml"

!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/sam-20140927.xml -o "sam-20140927.xml"
```



In [ ]:
#Housekeeping to load example files to current working directory    
!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/sam-20131228.xml -o "sam-20131228.xml"

!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/sam-20140927.xml -o "sam-20140927.xml"

In [ ]:
# Housekeeping to install python-xbrl 
!pip install python-xbrl    

# Housekeeping to import required modules 
from xbrl import XBRLParser, GAAP, GAAPSerializer, DEISerializer

# Housekeeping to import prettyprint (pprint)
import pprint

###Simple XBRL Parsing Workflow    

First parse the incoming `XRBL` file into a new `XBRL` basic object    



In [ ]:
xbrl_parser = XBRLParser()
xbrl = xbrl_parser.parse("sam-20131228.xml")

Then you can parse the document using different parsers


In [ ]:
gaap_obj = xbrl_parser.parseGAAP(xbrl, doc_date="20131228", context="current", ignore_errors=0)

Now we have a GAAP model object that has the GAAP parsed elements from the document.    

This model object supports the several different features including:    

* **context current year**, and **instance contexts** are supported. If available you can also get **previous quarter** information by number of days from doc date. Example: 90, 180, etc.    
* **Error handling**  
    0. raise exception for all parsing errors and halt parsing
    1. Supress all parsing errors and continue parsing
    2. Log all parsing errors and continue parsing    

You can serialize the GAAP model object into a serialized object acceptable for rending into a standard format such as JSON or HTTP API. 
   

In [ ]:
serializer = GAAPSerializer()
result = serializer.dump(gaap_obj)

You can also just view the data in the serialized object

In [ ]:
print(result)    

In [ ]:
# import pprint    

pprint.pprint(result)

You can apply various parsers to the base `XBRLParser` object to get different data than just `GAAP` data from the document. In addition as expected you can also create different serialized objects on the resulting parsed data object.

###Extracting DEI Data    


In [ ]:
dei_obj = xbrl_parser.parseDEI(xbrl)
serializer = DEISerializer()
result = serializer.dump(dei_obj)
pprint.pprint(result)

###Extracting Custom Data    


In [ ]:
custom_obj = xbrl_parser.parseCustom(xbrl)
pprint.pprint(custom_obj())



---



This notebook includes work from references in **SEC Edgar** data file selection and parsing as well as market data access from **Yahoo! Finance**.  Source articles and repositories are identified and linked throughout the workbook.      

**Emailprofessorpatrick@gmail.com** with suggestions for additions or improvements to this notebook.    
